## Подгрузим Aligner

In [4]:
import os
from nltk.tokenize import sent_tokenize
from lingtrain_aligner import preprocessor, splitter, aligner, resolver, reader, helper, vis_helper
import json
from copy import deepcopy
from tqdm import tqdm
from contextlib import redirect_stdout
import io
import warnings

In [2]:
def align_texts(text_es, text_ru):
    model_name='sentence_transformer_multilingual'
    
    i=0
    while True:
        db_path = f'logs/{i}.db'
        if os.path.isfile(db_path):
            try:
                os.unlink(db_path)
                break
            except:
                i+=1
        else:
            break
    
    text_es = sent_tokenize(text_es.replace('\xa0', ' '))
    text_ru = sent_tokenize(text_ru.replace('\xa0', ' '))
    with redirect_stdout(io.StringIO()) as f:
        warnings.simplefilter("ignore")
        aligner.fill_db(db_path, 'es', 'ru', text_es, text_ru)
        aligner.align_db(db_path,
                    model_name,
                    batch_size=50,
                    window=5,
                    batch_ids=[0,1],
                    save_pic=False,
                    embed_batch_size=50,
                    normalize_embeddings=True,
                    show_progress_bar=False)

        steps = 3
        batch_id = -1  #all batches
        for i in range(steps):
            conflicts, rest = resolver.get_all_conflicts(db_path, min_chain_length=2+i, max_conflicts_len=6*(i+1), batch_id=batch_id)
            resolver.resolve_all_conflicts(db_path, conflicts, 'sentence_transformer_multilingual', show_logs=False)
            if len(rest) == 0:
                break
    
    paragraphs_from, paragraphs_to, meta = reader.get_paragraphs(db_path, direction="from")
    return {f'sentence{i+1}': {'es': paragraphs_from[0][i], 'ru': paragraphs_to[0][i]} for i in range(len(paragraphs_from[0]))}

In [ ]:
final = {}
for file in os.listdir('materials/'):
    with open('materials/' + file, 'r', encoding='utf-8') as f:
        dct = json.load(f)
    for sent, info in tqdm(dct.items()):
        new = deepcopy(info)
        new['Country'] = file.split('.')[0].capitalize()
        new['sentences'] = align_texts(new['text_es'], new['text_ru'])
        final[f'{file[:3]}_{sent}'] = new

with open('final.json', 'w', encoding='utf-8') as f:
        json.dump(final, f, ensure_ascii=False, indent='\t')

In [ ]:
final